In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import jax_smi
jax_smi.initialise_tracking()

In [5]:
from saex.trainer_cache import main
main()

Creating SAE...
Loading model...


Some weights of the model checkpoint at gpt2 were not used when initializing FlaxGPT2Model: {('h', '7', 'attn', 'bias'), ('h', '5', 'attn', 'bias'), ('h', '8', 'attn', 'bias'), ('h', '9', 'attn', 'bias'), ('h', '2', 'attn', 'bias'), ('h', '6', 'attn', 'bias'), ('h', '3', 'attn', 'bias'), ('h', '4', 'attn', 'bias'), ('h', '11', 'attn', 'bias'), ('h', '1', 'attn', 'bias'), ('h', '10', 'attn', 'bias'), ('h', '0', 'attn', 'bias')}
- This IS expected if you are initializing FlaxGPT2Model from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing FlaxGPT2Model from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Loading dataset...


/home/neverix/.cache/pypoetry/virtualenvs/saex-U2at97x7-py3.12/lib/python3.12/site-packages/datasets/load.py:1461: FutureWarning: The repository for Skylion007/openwebtext contains custom code which must be executed to correctly load the dataset. You can inspect the repository content at https://hf.co/datasets/Skylion007/openwebtext
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this dataset from the next major release of `datasets`.
  warnings.warn(
wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

Training for 100000 iterations


wandb: Appending key for api.wandb.ai to your netrc file: /home/neverix/.netrc


Learning rate: 0.0006 warmed up for 128 iterations and cycled every 100000 iterations


Training SAE:   5%|▍         | 4861/100000 [05:05<30:48, 51.48it/s, dead=0.06188965, l0=44.59813, loss=0.006977533, loss_reconstruction=0.0022079418, loss_sparsity=34.645283, max_time_since_fired=4607.0, var_explained=0.9133086]        